### Create Parquet File for Taxi Zones

In [0]:
import pandas as pd
import io
import os
import requests

In [0]:
url = 'https://d37ci6vzurychx.cloudfront.net/misc/taxi+_zone_lookup.csv'

web_content = requests.get(url).content
pandas_dataframe = pd.read_csv(io.StringIO(web_content.decode('utf-8')))

In [0]:
display(pandas_dataframe)

In [0]:
# access_key = dbutils.secrets.get(scope = 'aws', key = 'access_key')
# secret_key = dbutils.secrets.get(scope = 'aws', key = 'secret_key')

access_key = os.getenv('AWS_ACCESS_KEY')
secret_key = os.getenv('AWS_SECRET_KEY')

BUCKET_NAME = 'dmitov-nyc-taxi'
PARQUET_DIRECTORY = 'parquet-taxi-zones'

spark_dataframe = spark.createDataFrame(pandas_dataframe)

# spark_dataframe.coalesce(1).write.mode('overwrite').format('parquet').save(
#     's3a://{}:{}@{}/{}'.format(access_key, secret_key, BUCKET_NAME, PARQUET_DIRECTORY)
# )

temporary_parquet_directory = '/FileStore/temp'

spark_dataframe.coalesce(1).write.parquet(temporary_parquet_directory)

files = dbutils.fs.ls(temporary_parquet_directory)

for file in files:
    if '.snappy.parquet' in file.name:
        dbutils.fs.cp(
          file.path,
          's3a://{}:{}@{}/{}/taxi_zones.snappy.parquet'.format(
              access_key,
              secret_key,
              BUCKET_NAME,
              PARQUET_DIRECTORY
          )
        )

dbutils.fs.rm(temporary_parquet_directory, recurse=True)

In [0]:
control_parquet_dataframe = spark.read.format('parquet').load(
    's3a://{}:{}@{}/{}/taxi_zones.snappy.parquet'.format(
        access_key,
        secret_key,
        BUCKET_NAME,
        PARQUET_DIRECTORY
    )
)

display(control_parquet_dataframe)